# Using Python for Plaid API
## Last Updated: 2022-05-26

---

### Reference
1. Using Plaid Postsman for no-code API exploration: https://www.postman.com/plaid-api/workspace/plaid/overview, use the instruction here: https://github.com/plaid/plaid-postman
2. Using Python for API: https://github.com/plaid/plaid-python

### Setup
Install plaid-python

### Code

In [2]:
# basic libraries
import plaid
from plaid.api import plaid_api
import json
from datetime import date
from datetime import datetime

***

#### >This is from Postman's code snippet

In [ ]:
import http.client
import json

conn = http.client.HTTPSConnection("sandbox.plaid.com")
payload = json.dumps({
  "client_id": "5e7651a8c2c21f00134cb8d9",
  "secret": "6db12841a970cdb3829ae87f465f90",
  "institution_id": "ins_3",
  "initial_products": [
    "auth"
  ],
  "options": {
    "webhook": "https://www.genericwebhookurl.com/webhook"
  }
})
headers = {
  'Content-Type': 'application/json'
}
conn.request("POST", "/sandbox/public_token/create", payload, headers)
res = conn.getresponse()
data = res.read()
data_json = json.loads(data.decode("utf-8"))
print(data_json["public_token"])

---

#### >This is from Official Plaid Python package
https://github.com/plaid/plaid-python

In [3]:
# Available environments are
# 'Production'
# 'Development'
# 'Sandbox'
configuration = plaid.Configuration(
    host=plaid.Environment.Sandbox,
    api_key={
        'clientId': "5e7651a8c2c21f00134cb8d9",
        'secret': "6db12841a970cdb3829ae87f465f90",
    }
)
# get clientID and secret from https://dashboard.plaid.com/team/keys 
# DO NOT UPLOAD api key values to public repo!

api_client = plaid.ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)

In [5]:
# Get access token, this is required to pull reports
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest
from plaid.model.products import Products
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest
pt_request = SandboxPublicTokenCreateRequest(
    institution_id="ins_3",
    initial_products=[Products('assets')]
)
pt_response = client.sandbox_public_token_create(pt_request)
# The generated public_token can now be
# exchanged for an access_token
exchange_request = ItemPublicTokenExchangeRequest(
    public_token=pt_response['public_token']
)
exchange_response = client.item_public_token_exchange(exchange_request)
access_token = exchange_response['access_token']

In [6]:
# Collect asset report token to fetch report
from plaid.model.asset_report_create_request import AssetReportCreateRequest
from plaid.model.asset_report_create_request_options import AssetReportCreateRequestOptions
from plaid.model.asset_report_user import AssetReportUser
# referenced from https://stackoverflow.com/questions/71761238/cant-get-plaid-link-to-open-using-javascript-and-python
# access_tokens is a list of Item access tokens.
# Note that the assets product must be enabled for all Items.
# All fields on the options object are optional.
request = AssetReportCreateRequest(
    access_tokens=[access_token],
    days_requested=60,
    options=AssetReportCreateRequestOptions(
        webhook='https://www.example.com',
        client_report_id='123',
        user=AssetReportUser(
            client_user_id='7f57eb3d2a9j6480121fx361',
            first_name='Jane',
            middle_name='Leah',
            last_name='Doe',
            ssn='123-45-6789',
            phone_number='(555) 123-4567',
            email='jane.doe@example.com',
        )
    )
)
response = client.asset_report_create(request)
asset_report_id = response['asset_report_id']
asset_report_token = response['asset_report_token']

In [8]:
# get asset report from token
from plaid.model.asset_report_get_request import AssetReportGetRequest
try:
    request = AssetReportGetRequest(asset_report_token=asset_report_token)
    response = client.asset_report_get(request)
    report = response['report']
except plaid.ApiException as e:
    response = json.loads(e.body)
    # check the code attribute of the error to determine the specific error
    if response['error_code'] == 'ITEM_LOGIN_REQUIRED':
        # the users' login information has changed, generate a public_token
        # for the user and initialize Link in update mode to
        # restore access to this user's data
        # see https://plaid.com/docs/api/#updating-items-via-link
        print("login error")
    else:
        print("error")

In [9]:
# Parse report to json
json_string = json.dumps(report.to_dict(),default=str)
# Using a JSON string to write to a file
with open('asset_report.json', 'w') as outfile:
    outfile.write(json_string)
json_string

'{"asset_report_id": "0f765260-2f34-4d69-9c60-4d797bd1cb93", "client_report_id": "123", "date_generated": "2022-05-26 22:36:34+00:00", "days_requested": 60.0, "user": {"client_user_id": "7f57eb3d2a9j6480121fx361", "email": "jane.doe@example.com", "first_name": "Jane", "last_name": "Doe", "middle_name": "Leah", "phone_number": "(555) 123-4567", "ssn": "123-45-6789"}, "items": [{"item_id": "qMob6r1yrrsk91XyKKa8CB4mMnX8M4fdL48mp", "institution_name": "Chase", "institution_id": "ins_3", "date_last_updated": "2022-05-26 22:36:32+00:00", "accounts": [{"account_id": "5Ra4XeMmeetx8qJQeeb5IBRoEnNDrgC3VDGpP", "balances": {"available": 100.0, "current": 110.0, "limit": null, "iso_currency_code": "USD", "unofficial_currency_code": null}, "mask": "0000", "name": "Plaid Checking", "official_name": "Plaid Gold Standard 0% Interest Checking", "type": "depository", "subtype": "checking", "days_available": 60.0, "transactions": [{"original_description": "SparkFun", "account_id": "5Ra4XeMmeetx8qJQeeb5IBR

In [10]:
# Use this to generate pdf report
from plaid.model.asset_report_pdf_get_request import AssetReportPDFGetRequest
pdf_request = AssetReportPDFGetRequest(asset_report_token=asset_report_token)
pdf = client.asset_report_pdf_get(pdf_request)
FILE = open('asset_report.pdf', 'wb')
FILE.write(pdf.read())
FILE.close()

---

#### >Additional Resources

In [ ]:
# This is test code to parse response
response = ... # type TransactionsGetResponse
# to_dict makes it first a python dictionary, and then we turn it into a string JSON.
json_string = json.dumps(response.to_dict())

In [ ]:
# Use this to explore the plaid module
dir(plaid.model)

Links & Docs:
1. Token Endspoints: https://plaid.com/docs/api/tokens/
2. Assets: https://plaid.com/docs/api/products/assets/#asset_reportget